In [1]:
%%capture
!pip install transformers
!pip install -U sentence-transformers
!pip install datasets
!pip install evaluate
!pip install googletrans==3.1.0a0

from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from googletrans import Translator
from sentence_transformers import SentenceTransformer, util
import sys
import os
import pandas as pd
import numpy as np



In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
tokenizer.pad_token = tokenizer.eos_token
if 'model' not in globals():
  model =  AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B", pad_token_id=tokenizer.eos_token_id)

In [3]:
if 'emb_model' not in globals():
  emb_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
dataset = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/dataset_covered_recipes.csv",sep=';')#, header=0, index_col = 0)

In [4]:
dataset = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/dataset_covered_recipes.csv",sep=';')#, header=0, index_col = 0)
risposta_utente = "oats,banana,milk"  #input("Hi! insert a recipe or ingredients: ") #meat
embedding_ing = emb_model.encode(risposta_utente)

EleutherAI/gpt-neo-2.7B text generation

In [5]:
my_ingredients = 'oats,banana,milk'
embedding_ing = emb_model.encode(my_ingredients)

In [6]:
import sys
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

dataset_sustainable =  dataset.loc[np.where(dataset['SUSTAINABILITY'].eq('HIGH'))]
dataset_sustainable.reset_index(inplace = True,drop = True)
embedding_recipes = emb_model.encode(dataset_sustainable['ingredients'])

# more sustainable
scores = []  
scores = util.pytorch_cos_sim(embedding_ing, embedding_recipes).flatten()
print(scores)
sorted = scores.argsort()[-3:]
print(scores[sorted])
high_results = dataset_sustainable.loc[sorted]

tensor([0.0759, 0.2650, 0.2650,  ..., 0.6454, 0.3984, 0.4800])
tensor([0.8304, 0.8824, 0.9423])


In [10]:
dataset_mid_sustainable =  dataset.loc[np.where(dataset['SUSTAINABILITY'].eq('MEDIUM'))]
dataset_mid_sustainable.reset_index(inplace = True,drop = True)
embedding_mid_sus_recipes = emb_model.encode(dataset_mid_sustainable['ingredients'])

# medium sustainable
mid_sus_scores = []
mid_sus_scores = util.pytorch_cos_sim(embedding_ing, embedding_mid_sus_recipes).flatten()
print(mid_sus_scores)
mid_sorted = mid_sus_scores.argsort()[-3:]
print(mid_sus_scores[mid_sorted])
mid_sus_results = dataset_mid_sustainable.loc[mid_sorted]

tensor([0.6187, 0.4644, 0.6124,  ..., 0.4748, 0.4394, 0.5109])
tensor([0.7735, 0.7783, 0.7811])


## High sustainable

In [11]:
from transformers import set_seed

set_seed(42)

In [12]:
title = high_results.iloc[2,1]
url = high_results.iloc[2, 2]
ingredients = high_results.iloc[2, 3]

display(high_results, title, url, ingredients)

,recipe_id,title,url,ingredients,SUSTAINABILITY
487,35477,Survival Necklace,http://www.food.com/recipe/survival-necklace-9...,"banana,oat meal",HIGH
1951,50516,Strawberry Banana Smoothie,http://www.kraftrecipes.com/recipes/strawberry...,"cow milk,banana,strawberry,oat meal",HIGH
1728,2818,Oatmeal and Bananas Breakfast (Baby Food),http://www.food.com/recipe/oatmeal-and-bananas...,"oat,cow milk,banana",HIGH


'Oatmeal and Bananas Breakfast (Baby Food)'

'http://www.food.com/recipe/oatmeal-and-bananas-breakfast-baby-food-52539'

'oat,cow milk,banana'

In [13]:
prompt = ['A recipe '+ingredients]

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text1 = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text1[0]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[{'generated_text': 'A recipe oat,cow milk,banana cream and pineapple juice with sweetened shredded coconut and fresh pineapple,tossed with a little grated ginger in a bowl made into a mold and'}]

In [14]:
prompt = ['What do you think about '+title+' it has high sustainability']

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text2 = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text2[0]

[{'generated_text': 'What do you think about Oatmeal and Bananas Breakfast (Baby Food) it has high sustainability.\n\nDo you like the idea of Organic Bananas, Oatmeal (Breakfast Cereal)'}]

In [15]:
generated_text_high_sustainable = str(generated_text1[0][0]['generated_text']) + '.' + str(generated_text2[0][0]['generated_text']) + ', you can find more on: ' + high_results.iloc[2, 2]
print(generated_text_high_sustainable.replace('\n', ''))

A recipe oat,cow milk,banana cream and pineapple juice with sweetened shredded coconut and fresh pineapple,tossed with a little grated ginger in a bowl made into a mold and.What do you think about Oatmeal and Bananas Breakfast (Baby Food) it has high sustainability.Do you like the idea of Organic Bananas, Oatmeal (Breakfast Cereal), you can find more on: http://www.food.com/recipe/oatmeal-and-bananas-breakfast-baby-food-52539


## Medium sustainable

In [19]:
title = mid_sus_results.iloc[2,1]
url = mid_sus_results.iloc[2, 2]
ingredients = mid_sus_results.iloc[2, 3]

display(mid_sus_results, title, url, ingredients)

,recipe_id,title,url,ingredients,SUSTAINABILITY
5837,23711,Oatmeal With Blueberries and Cream,http://www.food.com/recipe/oatmeal-with-bluebe...,"oat,cream,cow milk,blueberry",MEDIUM
8495,39484,Apple Butter Oatmeal,http://www.food.com/recipe/apple-butter-oatmea...,"oat,cow milk,butter",MEDIUM
1461,45835,Fruit and Malt Oat Shake,http://www.food.com/recipe/fruit-and-malt-oat-...,"oat,cow milk,banana,semi-sweet chocolate bakin...",MEDIUM


'Fruit and Malt Oat Shake'

'http://www.food.com/recipe/fruit-and-malt-oat-shake-463414'

'oat,cow milk,banana,semi-sweet chocolate baking chips'

In [20]:
prompt = ['A recipe '+ingredients]

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text1 = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text1[0]

[{'generated_text': 'A recipe oat,cow milk,banana,semi-sweet chocolate baking chips\n\nThe first thing I do every morning is run to the refrigerator to check for the banana. I like'}]

In [21]:
prompt = ['What do you think about '+title+'? it has medium sustainability']

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text2 = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text2[0]

[{'generated_text': 'What do you think about Fruit and Malt Oat Shake? it has medium sustainability score.\n\nI am trying to buy it a long time before a long time and I don’t have'}]

In [23]:
generated_test_medium_sustainable = generated_text_high_sustainable = str(generated_text1[0][0]['generated_text']) + '.' + str(generated_text2[0][0]['generated_text']) + ', you can find more on: ' + mid_sus_results.iloc[2, 2]
print(generated_text_high_sustainable.replace('\n', ''))

A recipe oat,cow milk,banana,semi-sweet chocolate baking chipsThe first thing I do every morning is run to the refrigerator to check for the banana. I like.What do you think about Fruit and Malt Oat Shake? it has medium sustainability score.I am trying to buy it a long time before a long time and I don’t have, you can find more on: http://www.food.com/recipe/fruit-and-malt-oat-shake-463414
